In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
import nltk,re

In [2]:
df=pd.read_pickle(r'C:\Users\goura\Documents\DS and ML Projects\8. Projects\Movie Recommendation System\processed.pkl')

In [3]:
df.head()[:2]

,id,title,genres,keywords,overview,cast,crew,combined
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],Adventure Fantasy Action ocean drugabuse exoti...


In [4]:
df=df[['id','title','combined']]

In [5]:
df.head()

,id,title,combined
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...


In [6]:
'''Stop Words and Other symbols and punctuation should be removed'''
for i in range(len(df)):
    df.combined[i]=df.combined[i].lower()
    words=nltk.word_tokenize(df.combined[i])
    words=[re.sub('[^a-zA-Z]'," ",word) for word in words if word not in stopwords.words('english')]
    df.combined[i]=' '.join(words)

<ipython-input-6-363e68ffcf94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.combined[i]=' '.join(words)
<ipython-input-6-363e68ffcf94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.combined[i]=df.combined[i].lower()


In [7]:
df.combined[1]

'adventure fantasy action ocean drugabuse exoticisland eastindiatradingcompany loveofone slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa   long believed dead   come back life headed edge earth turner elizabeth swann   nothing quite seems   johnnydepp orlandobloom keiraknightley stellanskarsg rd goreverbinski'

In [8]:
count_vect=CountVectorizer(max_features=10000)

In [9]:
vector=count_vect.fit_transform(df.combined).toarray()

In [10]:
vector.shape

(4809, 10000)

In [11]:
similarity_score=cosine_similarity(vector)
print(similarity_score)

[[1.         0.07071068 0.04774099 ... 0.0204765  0.02174427 0.        ]
 [0.07071068 1.         0.07595545 ... 0.02171861 0.         0.        ]
 [0.04774099 0.07595545 1.         ... 0.02199529 0.         0.        ]
 ...
 [0.0204765  0.02171861 0.02199529 ... 1.         0.06010829 0.03924588]
 [0.02174427 0.         0.         ... 0.06010829 1.         0.06251357]
 [0.         0.         0.         ... 0.03924588 0.06251357 1.        ]]


In [12]:
def recommend(movie_user):
    movie_index=df[df.title==movie_user].index.values[0]
    
    similar_movies=pd.DataFrame(
        enumerate(similarity_score[movie_index])).drop(0,axis='columns').sort_values(by=1,ascending=False)
    
    similar_movies['Names']=list(map(
        lambda x: str(np.squeeze(df[df.index==x]['title'].values)),similar_movies.index.values))
    similar_movies['id']=list(map(
        lambda x: int(np.squeeze(df[df.index==x]['id'].values)),similar_movies.index.values))
    return similar_movies.Names.values[:10]

In [13]:
recommend('Spider-Man')

array(['Spider-Man', 'Spider-Man 3', 'Spider-Man 2',
       'The Amazing Spider-Man 2', 'The Amazing Spider-Man',
       'Arachnophobia', 'Kick-Ass', '21 Jump Street', 'X-Men',
       'Small Soldiers'], dtype=object)

In [14]:
import pickle

In [15]:
pickle.dump(df,open('movie_list.pkl','wb'))
pickle.dump(similarity_score,open('similarity.pkl','wb'))

In [16]:
df.head()

,id,title,combined
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...
